## Inference and getting ready for deployment

Let’s check if our models work in inferences.

We only test one image and do a visual inspection of the results.
As already mentioned before, I did not provide a test set. 

This is the biggest open TODO.

Another important aspect would be how certain the prediction is. How high is the probability for the second candidate?
Many improvements are possible in problem definition and post-processing. 

### Comparison of three models


In [1]:
from fastcore.all import *
from fastai.vision.all import *

In [3]:
from fastai.learner import load_learner

# Load the FastAI Learner
learn_inf_tiny = load_learner("models/tiny.pkl")
learn_inf_base= load_learner("models/base.pkl")
learn_inf_resnet = load_learner("models/resnet.pkl")

In [12]:
learn_inf_tiny.predict("working/which_cheese_cleaned/which_cheese_first/which_cheese/Cantal/0c81aeec-c0a6-421e-844f-3e6e240885a8.jpg")

('Cantal',
 tensor(4),
 tensor([9.7780e-05, 9.0306e-06, 2.1395e-05, 1.0606e-05, 9.9840e-01, 1.2682e-07,
         4.7644e-04, 1.4753e-06, 9.9773e-06, 4.0509e-06, 2.3105e-05, 8.3267e-05,
         8.9159e-05, 2.2647e-06, 3.8224e-06, 4.5492e-07, 2.8718e-04, 2.2553e-06,
         7.6010e-07, 3.5029e-04, 2.3085e-07, 1.2567e-04]))

In [13]:
learn_inf_base.predict("working/which_cheese_cleaned/which_cheese_first/which_cheese/Cantal/0c81aeec-c0a6-421e-844f-3e6e240885a8.jpg")

('Cantal',
 tensor(4),
 tensor([1.5877e-06, 5.6175e-05, 1.3185e-06, 4.0135e-06, 9.9739e-01, 1.9972e-06,
         1.6469e-03, 1.1616e-05, 1.5650e-04, 2.0251e-05, 1.6810e-05, 3.3364e-04,
         1.2042e-05, 1.8571e-06, 7.5011e-06, 5.5109e-07, 1.8472e-04, 2.0955e-06,
         1.5077e-05, 8.5131e-05, 1.5477e-07, 4.9878e-05]))

In [14]:
learn_inf_resnet.predict("working/which_cheese_cleaned/which_cheese_first/which_cheese/Cantal/0c81aeec-c0a6-421e-844f-3e6e240885a8.jpg")

('Cantal',
 tensor(4),
 tensor([1.2273e-06, 1.0518e-04, 2.1162e-05, 9.5564e-07, 9.9687e-01, 5.3281e-06,
         2.2306e-03, 5.5073e-08, 9.4349e-05, 1.3493e-06, 2.2718e-04, 2.3397e-04,
         8.0347e-06, 5.4313e-06, 4.4896e-06, 8.3956e-07, 3.2568e-05, 1.5521e-05,
         2.5339e-06, 1.2194e-04, 1.5376e-05, 4.0610e-07]))

### Comparison of ONNX and Pytorch

We’ll require an `onnx` model at a later time. Let’s evaluate the `resnet` model’s prediction accuracy.

In [ ]:
!pip install onnx

In [6]:
import torch

In [7]:
model = learn_inf_resnet.model
dummy_input = torch.randn(1, 3, 256, 256)  # Use batch size 1 for export
torch.onnx.export(
    model, 
    dummy_input, 
    "model.onnx", 
    export_params=True, 
    opset_version=11, 
    do_constant_folding=True, 
    input_names=["input"], 
    output_names=["output"], 
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}  # Allow variable batch size
)


In [ ]:
!pip install onnxruntime numpy pillow torchvision

In [8]:
class_names = learn_inf_resnet.dls.vocab
print(class_names)  # List of class names

['Banon', 'Bleu d’Auvergne', 'Brie de Meaux', 'Camembert', 'Cantal', 'Chabichou du Poitou', 'Comté', 'Fourme d’Ambert', 'Gruyere', 'Livarot', 'Manchego', 'Mimolette', 'Munster', 'Neufchâtel', 'Pont-l’Évêque', 'Pélardon', 'Reblochon', 'Roquefort', 'Selles-sur-Cher', 'Tomme de Savoie', 'Valençay', 'Époisses de Bourgogne']


In [10]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
# Load ONNX model
session = ort.InferenceSession("model.onnx", providers=["CPUExecutionProvider"])

# Preprocessing function
def preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).numpy().astype(np.float32)  # Add batch dim and convert to NumPy
    return image

# Load and preprocess image
image_path = "working/which_cheese_cleaned/which_cheese_first/which_cheese/Cantal/0c81aeec-c0a6-421e-844f-3e6e240885a8.jpg"  # Replace with your image path
input_tensor = preprocess_image(image_path)

# Run inference
outputs = session.run(None, {"input": input_tensor})


import torch
outputs = session.run(None, {"input": input_tensor})[0]  # Raw logits
probabilities = torch.nn.functional.softmax(torch.tensor(outputs), dim=1)  # Convert to probabilities
predicted_class = torch.argmax(probabilities, dim=1).item()
# Get predicted class index and label
predicted_idx = np.argmax(probabilities)
predicted_label = class_names[predicted_idx]

print(f"Predicted Class: {predicted_label} (Confidence: {probabilities[0][predicted_idx]:.6f})")



Predicted Class: Cantal (Confidence: 0.971699)


The prediction is correct, but the confidence is slightly different. We will try it anyway in deployment.

## Deployment: Delivering an experience

Of course we want to share our model and not only by posting the source code on GitHub or hugging face. What we want is a live version of the model. Something users can experience.

You can deploy via cloud computing or on-device/edge computing.
The used technologies are different.

### Cloud based deployment

You cannot evaluate PyTorch ML models using simple JavaScript. A server runs a python backend, which provides an endpoint that is only doing the code of the previous section.

[Here](https://www.tanishq.ai/blog/posts/2021-11-16-gradio-huggingface.html) is a good tutorial how to get a simple setup running on hugging face with gradio.

I developed a webcam based app; the code is in the repo. [And the app is live](https://huggingface.co/spaces/dolind/whichcheese).

In the app you can select the convnext-base, convnext-tiny and the resnet model. All models are trained with 256px images. Just point the camera towards a cheese. 

I used the `Gradio` framework, popular in ML and featured on Hugging Face. The processing takes several milliseconds, despite being server based. The Gradio app offers no frame dropping. I try to include dynamic throttling to avoid frame congestion.

One thing I observed from this app is that the Convnext models have high numbers for the second and third best candidate. The app tries to predict a cheese when there is no cheese present. Two points worth to examine.

### Edge based deployment

The issue with edge-based deployment is python. Python is by default not available on mobile. And because of secure concerns, it is becoming more and more complex to run a full blown linux with a python installation.

The other two ways are mobile apps and browser-based inference. We limit ourselves to browser based, because this is accessible via desktop and mobile.

ONNX format is necessary for web browser model deployment. 
At the end of this, we will evaluate if inferring `onnx` gives us the same probability. 
Due to differences in preprocessing between fast.ai and manual methods, some variations may occur.

Using my knowledge of web development, I constructed a basic [app](https://www.storymelange.com/cheese_classifier/) that does inferring the resnet model.

My impressions were that the results were less good. But, the startup and inference time were acceptable and comparable to the python app.
This is a point worth to examine once I have defined a proper test set.

## The End

This was my first basic study in low level ML. I dabbled in pose recognition before and did work manage AI projects. I’m very impressed with the progress these tools have made.

If you have enjoyed the read, come back for the next project. We will revisit a recipe classification app, which I programmed in 2021 and which we will improve with AI.

### Links

[Github Repo](https://github.com/dolind/cheese_classifier)

[Javascript App](https://www.storymelange.com/cheese_classifier/) 

[Gradio App](https://huggingface.co/spaces/dolind/whichcheese)